In [32]:
#MERGING FPL DATA AND UNDERSTAT DATA 
import os
import pandas as pd

# Step 1: Create the merged_data folder structure
merged_data_dir = 'merged_data'
seasons = ['2021-22', '2022-23', '2023-24']
positions = ['FWD', 'MID', 'DEF', 'GK']

# Create the merged_data directory with subdirectories for each season and position
for season in seasons:
    season_dir = os.path.join(merged_data_dir, season)
    os.makedirs(season_dir, exist_ok=True)
    
    for position in positions:
        position_dir = os.path.join(season_dir, position)
        os.makedirs(position_dir, exist_ok=True)

# Step 2: Define paths to fpl_gw_data and understat_data directories
fpl_gw_data_dir = 'fpl_gw_data'
understat_data_dir = 'understat_data'

# Step 3: Process each CSV file in fpl_gw_data
for season in seasons:
    fpl_season_dir = os.path.join(fpl_gw_data_dir, season)
    understat_season_dir = os.path.join(understat_data_dir, season)
    
    if os.path.isdir(fpl_season_dir) and os.path.isdir(understat_season_dir):
        for fpl_csv in os.listdir(fpl_season_dir):
            if fpl_csv.endswith('.csv'):
                fpl_csv_path = os.path.join(fpl_season_dir, fpl_csv)
                
                # Read the fpl_gw_data CSV
                fpl_df = pd.read_csv(fpl_csv_path)
                
                # Get player name and position
                player_name = fpl_df['name'].iloc[0]  
                # Capitalize first and last name for matching
                player_name_capitalized = '_'.join([name.capitalize() for name in player_name.split()])
                player_position = fpl_df['position'].iloc[0]
                
                # Find the corresponding understat file based on the player name
                understat_csv_pattern = f"{player_name_capitalized}_"
                matching_files = [f for f in os.listdir(understat_season_dir) if f.startswith(understat_csv_pattern)]
                
                if matching_files:
                    understat_csv_path = os.path.join(understat_season_dir, matching_files[0])
                    
                    # Read the understat CSV
                    understat_df = pd.read_csv(understat_csv_path)
                    
                    # Merge fpl_gw_data with understat_data based on matching kickoff_time and date
                    fpl_df['kickoff_time'] = pd.to_datetime(fpl_df['kickoff_time']).dt.date
                    understat_df['date'] = pd.to_datetime(understat_df['date']).dt.date
                    
                    merged_df = pd.merge(fpl_df, understat_df, left_on='kickoff_time', right_on='date', how='inner')
                    
                    # Save the merged file in the correct position folder in merged_data
                    merged_output_dir = os.path.join(merged_data_dir, season, player_position)
                    merged_output_path = os.path.join(merged_output_dir, fpl_csv)
                    
                    merged_df.to_csv(merged_output_path, index=False)
                    print(f"Merged and saved {fpl_csv} to {merged_output_dir}")
                else:
                    print(f"No matching file found for {player_name} in season {season} understat_data.")
    else:
        print(f"Season folder {season} does not exist in both fpl_gw_data and understat_data.")


Merged and saved Miguel_Almirón.csv to merged_data/2021-22/MID
No matching file found for Isaac Success Ajayi in season 2021-22 understat_data.
No matching file found for Joseph Hungbo in season 2021-22 understat_data.
No matching file found for Patrik Gunnarsson in season 2021-22 understat_data.
Merged and saved Nathan_Aké.csv to merged_data/2021-22/DEF
Merged and saved Jamie_Vardy.csv to merged_data/2021-22/FWD
Merged and saved Lukas_Rupp.csv to merged_data/2021-22/MID
Merged and saved Edo_Kayembe.csv to merged_data/2021-22/MID
Merged and saved Vitaly_Janelt.csv to merged_data/2021-22/MID
Merged and saved Andreas_Christensen.csv to merged_data/2021-22/DEF
No matching file found for Morgan Gibbs-White in season 2021-22 understat_data.
Merged and saved Jonjo_Shelvey.csv to merged_data/2021-22/MID
No matching file found for Dara Costelloe in season 2021-22 understat_data.
Merged and saved Josh_Benson.csv to merged_data/2021-22/MID
Merged and saved Bryan_Gil_Salvatierra.csv to merged_dat

In [33]:
#RESTRUCTURING THE FWD FOLDER DATA
import os
import pandas as pd

# Define the path to the FWD folder in each season
merged_data_dir = 'merged_data'
seasons = ['2021-22', '2022-23', '2023-24']
fwd_folder = 'FWD'

# Columns to drop
columns_to_drop = ['position_x', 'kickoff_time', 'time']

# Column order as specified, including 'starts' as optional
desired_column_order = [
    'name', 'team', 'opponent_team', 'date', 'was_home', 'position_y', 'minutes', 'goals', 'xG', 'assists', 'xA',
    'total_points', 'shots', 'key_passes', 'ict_index', 'bonus', 'starts'
]

# Loop through each season and process the FWD folder
for season in seasons:
    fwd_dir = os.path.join(merged_data_dir, season, fwd_folder)
    
    if os.path.isdir(fwd_dir):
        for csv_file in os.listdir(fwd_dir):
            if csv_file.endswith('.csv'):
                file_path = os.path.join(fwd_dir, csv_file)
                
                # Read the CSV file
                df = pd.read_csv(file_path)
                
                # Step 1: Drop the unnecessary columns
                df = df.drop(columns=[col for col in columns_to_drop if col in df.columns], errors='ignore')
                
                # Step 2: Convert 'starts' column to boolean (True/False) if it exists
                if 'starts' in df.columns:
                    df['starts'] = df['starts'].apply(lambda x: True if x == 1 else False)
                else:
                    print(f"'starts' column not found in {csv_file}, skipping the conversion.")
                
                # Step 3: Reorder the columns, but first check for missing columns
                existing_columns = [col for col in desired_column_order if col in df.columns]
                df = df[existing_columns]
                
                # Step 4: Rename 'position_y' to 'position' if 'position_y' exists
                if 'position_y' in df.columns:
                    df = df.rename(columns={'position_y': 'position'})
                
                # Save the modified CSV file back
                df.to_csv(file_path, index=False)
                print(f"Processed and saved {csv_file} in {fwd_dir}")



'starts' column not found in Jamie_Vardy.csv, skipping the conversion.
Processed and saved Jamie_Vardy.csv in merged_data/2021-22/FWD
'starts' column not found in Jordan_Ayew.csv, skipping the conversion.
Processed and saved Jordan_Ayew.csv in merged_data/2021-22/FWD
'starts' column not found in Joshua_King.csv, skipping the conversion.
Processed and saved Joshua_King.csv in merged_data/2021-22/FWD
'starts' column not found in Odsonne_Edouard.csv, skipping the conversion.
Processed and saved Odsonne_Edouard.csv in merged_data/2021-22/FWD
'starts' column not found in Harry_Kane.csv, skipping the conversion.
Processed and saved Harry_Kane.csv in merged_data/2021-22/FWD
'starts' column not found in Cameron_Archer.csv, skipping the conversion.
Processed and saved Cameron_Archer.csv in merged_data/2021-22/FWD
'starts' column not found in Michael_Obafemi.csv, skipping the conversion.
Processed and saved Michael_Obafemi.csv in merged_data/2021-22/FWD
'starts' column not found in Lewis_Richard

In [34]:
#RESTRUCTURING THE MID FOLDER 
import os
import pandas as pd

# Define the path to the MID folder in each season
merged_data_dir = 'merged_data'
seasons = ['2021-22', '2022-23', '2023-24']
mid_folder = 'MID'

# Columns to drop
columns_to_drop = ['position_x', 'kickoff_time', 'time']

# Column order as specified for the MID folder
desired_column_order = [
    'name', 'team', 'opponent_team', 'date', 'was_home', 'position_y', 'minutes', 'goals', 'xG', 'assists', 'xA',
    'total_points', 'shots', 'key_passes', 'ict_index', 'bonus', 'clean_sheets', 'goals_conceded', 'starts'
]

# Loop through each season and process the MID folder
for season in seasons:
    mid_dir = os.path.join(merged_data_dir, season, mid_folder)
    
    if os.path.isdir(mid_dir):
        for csv_file in os.listdir(mid_dir):
            if csv_file.endswith('.csv'):
                file_path = os.path.join(mid_dir, csv_file)
                
                # Read the CSV file
                df = pd.read_csv(file_path)
                
                # Step 1: Drop the unnecessary columns
                df = df.drop(columns=[col for col in columns_to_drop if col in df.columns], errors='ignore')
                
                # Step 2: Convert 'starts' column to boolean (True/False) if it exists
                if 'starts' in df.columns:
                    df['starts'] = df['starts'].apply(lambda x: True if x == 1 else False)
                else:
                    print(f"'starts' column not found in {csv_file}, skipping the conversion.")
                
                # Step 3: Reorder the columns, but first check for missing columns
                existing_columns = [col for col in desired_column_order if col in df.columns]
                df = df[existing_columns]
                
                # Step 4: Rename 'position_y' to 'position' if 'position_y' exists
                if 'position_y' in df.columns:
                    df = df.rename(columns={'position_y': 'position'})
                
                # Save the modified CSV file back
                df.to_csv(file_path, index=False)
                print(f"Processed and saved {csv_file} in {mid_dir}")


'starts' column not found in Miguel_Almirón.csv, skipping the conversion.
Processed and saved Miguel_Almirón.csv in merged_data/2021-22/MID
'starts' column not found in Lukas_Rupp.csv, skipping the conversion.
Processed and saved Lukas_Rupp.csv in merged_data/2021-22/MID
'starts' column not found in Edo_Kayembe.csv, skipping the conversion.
Processed and saved Edo_Kayembe.csv in merged_data/2021-22/MID
'starts' column not found in Vitaly_Janelt.csv, skipping the conversion.
Processed and saved Vitaly_Janelt.csv in merged_data/2021-22/MID
'starts' column not found in Jonjo_Shelvey.csv, skipping the conversion.
Processed and saved Jonjo_Shelvey.csv in merged_data/2021-22/MID
'starts' column not found in Josh_Benson.csv, skipping the conversion.
Processed and saved Josh_Benson.csv in merged_data/2021-22/MID
'starts' column not found in Bryan_Gil_Salvatierra.csv, skipping the conversion.
Processed and saved Bryan_Gil_Salvatierra.csv in merged_data/2021-22/MID
'starts' column not found in G

In [35]:
import os
import pandas as pd

# RESTRUCTURING THE DEF FOLDER
merged_data_dir = 'merged_data'
seasons = ['2021-22', '2022-23', '2023-24']
def_folder = 'DEF'

# Columns to drop
columns_to_drop = ['position_x', 'kickoff_time', 'time', 'own_goals']

# Column order as specified for the DEF folder
desired_column_order = [
    'name', 'team', 'opponent_team', 'date', 'was_home', 'position_y', 'minutes', 'goals', 'xG', 'assists', 'xA',
    'total_points', 'shots', 'key_passes', 'ict_index', 'bonus', 'clean_sheets', 'goals_conceded', 'expected_goals_conceded', 'starts'
]

# Loop through each season and process the DEF folder
for season in seasons:
    def_dir = os.path.join(merged_data_dir, season, def_folder)
    
    if os.path.isdir(def_dir):
        for csv_file in os.listdir(def_dir):
            if csv_file.endswith('.csv'):
                file_path = os.path.join(def_dir, csv_file)
                
                # Read the CSV file
                df = pd.read_csv(file_path)
                
                # Step 1: Drop the unnecessary columns
                df = df.drop(columns=[col for col in columns_to_drop if col in df.columns], errors='ignore')
                
                # Step 2: Convert 'starts' column to boolean (True/False) if it exists
                if 'starts' in df.columns:
                    df['starts'] = df['starts'].apply(lambda x: True if x == 1 else False)
                else:
                    print(f"'starts' column not found in {csv_file}, skipping the conversion.")
                
                # Step 3: Reorder the columns, but first check for missing columns
                existing_columns = [col for col in desired_column_order if col in df.columns]
                df = df[existing_columns]
                
                # Step 4: Rename 'position_y' to 'position' if 'position_y' exists
                if 'position_y' in df.columns:
                    df = df.rename(columns={'position_y': 'position'})
                
                # Save the modified CSV file back
                df.to_csv(file_path, index=False)
                print(f"Processed and saved {csv_file} in {def_dir}")


'starts' column not found in Nathan_Aké.csv, skipping the conversion.
Processed and saved Nathan_Aké.csv in merged_data/2021-22/DEF
'starts' column not found in Andreas_Christensen.csv, skipping the conversion.
Processed and saved Andreas_Christensen.csv in merged_data/2021-22/DEF
'starts' column not found in Pablo_Marí.csv, skipping the conversion.
Processed and saved Pablo_Marí.csv in merged_data/2021-22/DEF
'starts' column not found in Tariq_Lamptey.csv, skipping the conversion.
Processed and saved Tariq_Lamptey.csv in merged_data/2021-22/DEF
'starts' column not found in Adam_Webster.csv, skipping the conversion.
Processed and saved Adam_Webster.csv in merged_data/2021-22/DEF
'starts' column not found in Nathaniel_Phillips.csv, skipping the conversion.
Processed and saved Nathaniel_Phillips.csv in merged_data/2021-22/DEF
'starts' column not found in Jarrad_Branthwaite.csv, skipping the conversion.
Processed and saved Jarrad_Branthwaite.csv in merged_data/2021-22/DEF
'starts' column 

In [36]:
import os
import pandas as pd

# Define the path to the GK folder in each season
merged_data_dir = 'merged_data'
seasons = ['2021-22', '2022-23', '2023-24']
gk_folder = 'GK'

# Columns to drop
columns_to_drop = ['position_x', 'kickoff_time', 'time', 'own_goals', 'goals', 'shots', 'xG', 'assists', 'key_passes', 'position_y']

# Column order as specified for the GK folder
desired_column_order = [
    'name', 'team', 'opponent_team', 'date', 'was_home', 'minutes', 'goals_conceded', 'expected_goals_conceded',
    'saves', 'penalties_saved', 'total_points', 'bonus', 'clean_sheets', 'xA', 'starts'
]

# Loop through each season and process the GK folder
for season in seasons:
    gk_dir = os.path.join(merged_data_dir, season, gk_folder)
    
    if os.path.isdir(gk_dir):
        for csv_file in os.listdir(gk_dir):
            if csv_file.endswith('.csv'):
                file_path = os.path.join(gk_dir, csv_file)
                
                # Read the CSV file
                df = pd.read_csv(file_path)
                
                # Step 1: Drop the unnecessary columns
                df = df.drop(columns=[col for col in columns_to_drop if col in df.columns], errors='ignore')
                
                # Step 2: Convert 'starts' and 'clean_sheets' columns to boolean (True/False) if they exist
                if 'starts' in df.columns:
                    df['starts'] = df['starts'].apply(lambda x: True if x == 1 else False)
                else:
                    print(f"'starts' column not found in {csv_file}, skipping the conversion.")
                
                if 'clean_sheets' in df.columns:
                    df['clean_sheets'] = df['clean_sheets'].apply(lambda x: True if x == 1 else False)
                else:
                    print(f"'clean_sheets' column not found in {csv_file}, skipping the conversion.")
                
                # Step 3: Reorder the columns, but first check for missing columns
                existing_columns = [col for col in desired_column_order if col in df.columns]
                df = df[existing_columns]
                
                # Save the modified CSV file back
                df.to_csv(file_path, index=False)
                print(f"Processed and saved {csv_file} in {gk_dir}")


'starts' column not found in Freddie_Woodman.csv, skipping the conversion.
Processed and saved Freddie_Woodman.csv in merged_data/2021-22/GK
'starts' column not found in Martin_Dubravka.csv, skipping the conversion.
Processed and saved Martin_Dubravka.csv in merged_data/2021-22/GK
'starts' column not found in Hugo_Lloris.csv, skipping the conversion.
Processed and saved Hugo_Lloris.csv in merged_data/2021-22/GK
'starts' column not found in Angus_Gunn.csv, skipping the conversion.
Processed and saved Angus_Gunn.csv in merged_data/2021-22/GK
'starts' column not found in Illan_Meslier.csv, skipping the conversion.
Processed and saved Illan_Meslier.csv in merged_data/2021-22/GK
'starts' column not found in Lukasz_Fabianski.csv, skipping the conversion.
Processed and saved Lukasz_Fabianski.csv in merged_data/2021-22/GK
'starts' column not found in Willy_Caballero.csv, skipping the conversion.
Processed and saved Willy_Caballero.csv in merged_data/2021-22/GK
'starts' column not found in Darr